This notebook will be for data cleaning

In [1]:
import pandas as pd

In [3]:
mta = pd.read_csv('https://data.ny.gov/api/views/ypri-5iwa/rows.csv?accessType=DOWNLOAD&sorting=true')

In [4]:
mta.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/31/2019 12:00:00 AM,00:00:00,REGULAR,7183242,2433142
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/31/2019 12:00:00 AM,04:00:00,REGULAR,7183258,2433149
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/31/2019 12:00:00 AM,08:00:00,REGULAR,7183278,2433176
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/31/2019 12:00:00 AM,12:00:00,REGULAR,7183393,2433262
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/31/2019 12:00:00 AM,16:00:00,REGULAR,7183572,2433312


In [5]:
#removing unnecessary columns
mta.drop(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME','DIVISION','DESC'],
        inplace=True,
        axis=1)

In [11]:
# checking for null data (horay there isn't any)
mta.isna().sum()

DATE       0
TIME       0
ENTRIES    0
EXITS      0
dtype: int64

In [12]:
#simplifying the date column (removing timestamp)
mta['DATE'].()

0           08/31/2019 12:00:00 AM
1           08/31/2019 12:00:00 AM
2           08/31/2019 12:00:00 AM
3           08/31/2019 12:00:00 AM
4           08/31/2019 12:00:00 AM
                     ...          
16233622    01/01/2021 12:00:00 AM
16233623    01/01/2021 12:00:00 AM
16233624    01/01/2021 12:00:00 AM
16233625    01/01/2021 12:00:00 AM
16233626    01/01/2021 12:00:00 AM
Name: DATE, Length: 16233627, dtype: object